In [2]:
from bs4 import BeautifulSoup
# import pandas as pd
from datetime import datetime, timedelta
from itertools import combinations
import json
import requests
import math
from typing import Optional, List, Dict

from pydantic import BaseModel
from typing import Mapping
import numpy as np
import polars as pl
import hvplot.polars
import pandas as pd
import altair as alt
import math

from shared.utils import formalize_data_with_str
from shared.random_rules import RandomRules

In [3]:
rr = RandomRules(
  TOTAL_NUMBERS = 35,
  CHOOSE = 5,
  MAX_PRIMES = 3,
  N_VALUES=["1", "2", "3"],
  # Rangos de valores de las bolillas
  MIN_B_VALUES = [1, 2, 3, 4, 5],
  MAX_B_VALUES = [31, 32, 33, 34, 35]
)

In [4]:
url = 'https://resultados.latinka.com.pe/i.do?m=historico&t=0&s=164'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

tables = soup.select('table')
rows = tables[1].find_all('tr')

In [5]:
lst = []
post_str_b1 = None
post_str_blast = None
for row in rows:
  cols = [data.text.strip() for data in row.find_all('td')]
  
  id, bolillas_int, str_combo = formalize_data_with_str(bolillas_str=cols[2])
  cols.append(id)
  cols.append(bolillas_int)
  cols.extend([b for b in bolillas_int])
  cols.extend(str_combo)
  cols.append(post_str_b1)
  cols.append(post_str_blast)

  is_rare, reason = rr.is_rare_combination(bolillas_int, id)
  cols.append(is_rare)
  cols.append(reason)

  cols.append(rr.jumps_map(bolillas_int))
  cols.append(rr.unijump(bolillas_int))
  
  lst.append(cols)

  post_str_b1 = str_combo[0]
  post_str_blast = str_combo[-1]

with open('./gn_cahe_v3.json', 'w') as file: 
  json.dump(lst, file)

dfRows = pl.DataFrame(
  lst,
  schema=[
    'fecha', 'sorteo', 'bolillas',
    'id', 'combo', 'b1', 'b2', 'b3', 'b4', 'b5',
    'str_b1', 'str_b2', 'str_b3', 'str_b4', 'str_b5',
    'post_str_b1', 'post_str_blast',
    'is_rare_combination', 'reason',
    "jumps_map", "unijump"
  ],
  schema_overrides={
    'combo': pl.Array(pl.UInt8, 5),
    'b1': pl.UInt8,
    'b2': pl.UInt8,
    'b3': pl.UInt8,
    'b4': pl.UInt8,
    'b5': pl.UInt8,
    'b6': pl.UInt8,
    'reason': pl.UInt8
  },
  orient='row'
)

TOTAL_LINES = len(dfRows)
print(f'{TOTAL_LINES=}')
display(dfRows.head(10))

TOTAL_LINES=7723


fecha,sorteo,bolillas,id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""07/04/2025""","""4179""","""08 21 22 24 07""","""0708212224""","[7, 8, … 24]",7,8,21,22,24,"""07""","""08""","""21""","""22""","""24""",null,null,true,16,"""01_13_01_02""",37
"""06/04/2025""","""4178""","""07 35 24 34 14""","""0714243435""","[7, 14, … 35]",7,14,24,34,35,"""07""","""14""","""24""","""34""","""35""","""07""","""24""",true,8,"""07_10_10_01""",-6
"""05/04/2025""","""4177""","""34 33 15 05 16""","""0515163334""","[5, 15, … 34]",5,15,16,33,34,"""05""","""15""","""16""","""33""","""34""","""07""","""35""",true,16,"""10_01_17_01""",-57
"""04/04/2025""","""4176""","""27 24 32 30 35""","""2427303235""","[24, 27, … 35]",24,27,30,32,35,"""24""","""27""","""30""","""32""","""35""","""05""","""34""",true,24,"""03_03_02_03""",3
"""03/04/2025""","""4175""","""11 06 05 20 12""","""0506111220""","[5, 6, … 20]",5,6,11,12,20,"""05""","""06""","""11""","""12""","""20""","""24""","""35""",true,16,"""01_05_01_08""",19
"""02/04/2025""","""4174""","""26 14 24 11 17""","""1114172426""","[11, 14, … 26]",11,14,17,24,26,"""11""","""14""","""17""","""24""","""26""","""05""","""20""",true,24,"""03_03_07_02""",-13
"""01/04/2025""","""4173""","""12 25 33 13 08""","""0812132533""","[8, 12, … 33]",8,12,13,25,33,"""08""","""12""","""13""","""25""","""33""","""11""","""26""",false,0,"""04_01_12_08""",-29
"""31/03/2025""","""4172""","""07 22 34 26 16""","""0716222634""","[7, 16, … 34]",7,16,22,26,34,"""07""","""16""","""22""","""26""","""34""","""08""","""33""",false,0,"""09_06_04_08""",5
"""30/03/2025""","""4171""","""35 05 15 29 03""","""0305152935""","[3, 5, … 35]",3,5,15,29,35,"""03""","""05""","""15""","""29""","""35""","""07""","""34""",true,2,"""02_10_14_06""",-8


In [6]:
import holoviews as hv
def plot_post_col (col: str, post_col: str, test_value: str):
  df_viz = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      post_col
    ).agg(pl.count(post_col).alias('count')) \
    .sort("count", descending=True)
  
  
  df_10_last = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .head(10)
  
  # Agrega una columna que sea el porcentaje de la frecuencia relativa
  total = df_viz['count'].sum()
  df_viz = df_viz.with_columns(
    ( (pl.col('count') / total) * 100 ).alias('percent')
  )
  mean_percent = df_viz['percent'].mean()

  chart = df_viz.hvplot.bar(
    x=post_col,
    y='percent'
  )

  chart = chart * hv.HLine(mean_percent).opts(color='red')
  chart.opts(default_tools=["pan"])

  df_jumps_map = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      'jumps_map'
    ).agg(pl.count('jumps_map').alias('count')) \
    .sort("count", descending=True)
  
  # chart_unijump = df_jumps_map.hvplot.bar(
  #   x='jumps_map',
  #   y='count'
  # )

  display(df_viz)
  display(chart)
  display(df_10_last)
  display(df_jumps_map)

  lst_post_str_b = df_viz.filter(pl.col('percent') >= mean_percent)[post_col].to_list()
  lst_jumps_map = df_jumps_map['jumps_map'].to_list()
  return lst_post_str_b, lst_jumps_map

In [14]:
possible_b1, exclude_jump_maps_b1 = plot_post_col(col='str_b1', post_col='post_str_b1', test_value="07")

post_str_b1,count,percent
str,u32,f64
"""01""",78,15.384615
"""03""",60,11.83432
"""04""",56,11.045365
"""02""",54,10.650888
"""05""",44,8.678501
…,…,…
"""17""",3,0.591716
"""16""",2,0.394477
"""21""",1,0.197239


:Overlay
   .Bars.I  :Bars   [post_str_b1]   (percent)
   .HLine.I :HLine   [x,y]

fecha,sorteo,bolillas,id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""06/04/2025""","""4178""","""07 35 24 34 14""","""0714243435""","[7, 14, … 35]",7,14,24,34,35,"""07""","""14""","""24""","""34""","""35""","""07""","""24""",true,8,"""07_10_10_01""",-6
"""31/03/2025""","""4172""","""07 22 34 26 16""","""0716222634""","[7, 16, … 34]",7,16,22,26,34,"""07""","""16""","""22""","""26""","""34""","""08""","""33""",false,0,"""09_06_04_08""",5
"""18/03/2025""","""4159""","""07 34 16 12 21""","""0712162134""","[7, 12, … 34]",7,12,16,21,34,"""07""","""12""","""16""","""21""","""34""","""01""","""22""",false,0,"""05_04_05_13""",5
"""13/03/2025""","""4154""","""17 30 18 07 19""","""0717181930""","[7, 17, … 30]",7,17,18,19,30,"""07""","""17""","""18""","""19""","""30""","""03""","""35""",true,24,"""10_01_01_11""",1
"""10/02/2025""","""4123""","""32 07 19 25 21""","""0719212532""","[7, 19, … 32]",7,19,21,25,32,"""07""","""19""","""21""","""25""","""32""","""08""","""32""",false,0,"""12_02_04_07""",-11
"""05/01/2025""","""4087""","""28 19 07 23 20""","""0719202328""","[7, 19, … 28]",7,19,20,23,28,"""07""","""19""","""20""","""23""","""28""","""01""","""34""",false,0,"""12_01_03_05""",-13
"""20/12/2024""","""4071""","""08 22 23 07 27""","""0708222327""","[7, 8, … 27]",7,8,22,23,27,"""07""","""08""","""22""","""23""","""27""","""05""","""34""",true,16,"""01_14_01_04""",42
"""29/11/2024""","""4050""","""13 26 29 25 07""","""0713252629""","[7, 13, … 29]",7,13,25,26,29,"""07""","""13""","""25""","""26""","""29""","""11""","""35""",false,0,"""06_12_01_03""",30
"""20/11/2024""","""4041""","""28 10 07 15 30""","""0710152830""","[7, 10, … 30]",7,10,15,28,30,"""07""","""10""","""15""","""28""","""30""","""12""","""34""",false,0,"""03_05_13_02""",-25


jumps_map,count
str,u32
"""04_11_05_06""",2
"""04_01_09_05""",2
"""08_01_01_18""",2
"""05_05_13_05""",2
"""01_02_09_07""",2
…,…
"""05_13_04_06""",1
"""13_02_01_05""",1
"""02_03_14_06""",1


In [15]:
possible_blast, exclude_jump_maps_blast = plot_post_col(col='str_b5', post_col='post_str_blast', test_value="24")

post_str_blast,count,percent
str,u32,f64
"""34""",34,14.345992
"""33""",31,13.080169
"""32""",28,11.814346
"""35""",26,10.970464
"""27""",16,6.751055
…,…,…
"""23""",2,0.843882
"""10""",1,0.421941
"""15""",1,0.421941


:Overlay
   .Bars.I  :Bars   [post_str_blast]   (percent)
   .HLine.I :HLine   [x,y]

fecha,sorteo,bolillas,id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""04/03/2025""","""4145""","""24 09 14 16 22""","""0914162224""","[9, 14, … 24]",9,14,16,22,24,"""09""","""14""","""16""","""22""","""24""","""06""","""22""",true,16,"""05_02_06_02""",-15
"""02/02/2025""","""4115""","""01 13 10 06 24""","""0106101324""","[1, 6, … 24]",1,6,10,13,24,"""01""","""06""","""10""","""13""","""24""","""17""","""33""",false,0,"""05_04_03_11""",9
"""31/12/2024""","""4082""","""01 13 16 24 15""","""0113151624""","[1, 13, … 24]",1,13,15,16,24,"""01""","""13""","""15""","""16""","""24""","""17""","""35""",false,0,"""12_02_01_08""",-1
"""24/12/2024""","""4075""","""07 03 24 18 15""","""0307151824""","[3, 7, … 24]",3,7,15,18,24,"""03""","""07""","""15""","""18""","""24""","""05""","""28""",false,0,"""04_08_03_06""",17
"""05/12/2024""","""4056""","""15 23 05 24 08""","""0508152324""","[5, 8, … 24]",5,8,15,23,24,"""05""","""08""","""15""","""23""","""24""","""05""","""27""",false,0,"""03_07_08_01""",-5
"""19/11/2024""","""4040""","""07 10 21 17 24""","""0710172124""","[7, 10, … 24]",7,10,17,21,24,"""07""","""10""","""17""","""21""","""24""","""07""","""30""",true,16,"""03_07_04_03""",9
"""18/11/2024""","""4039""","""01 07 19 23 24""","""0107192324""","[1, 7, … 24]",1,7,19,23,24,"""01""","""07""","""19""","""23""","""24""","""07""","""24""",false,0,"""06_12_04_01""",19
"""12/10/2024""","""4002""","""03 02 07 21 24""","""0203072124""","[2, 3, … 24]",2,3,7,21,24,"""02""","""03""","""07""","""21""","""24""","""06""","""31""",false,0,"""01_04_14_03""",-28
"""11/10/2024""","""4001""","""01 07 14 05 24""","""0105071424""","[1, 5, … 24]",1,5,7,14,24,"""01""","""05""","""07""","""14""","""24""","""02""","""24""",true,16,"""04_02_07_10""",-9


jumps_map,count
str,u32
"""09_01_04_04""",2
"""01_07_05_01""",2
"""01_05_05_02""",2
"""11_06_01_05""",1
"""03_04_01_01""",1
…,…
"""01_02_06_14""",1
"""17_02_03_01""",1
"""04_07_05_07""",1


In [16]:
b_count = rr.CHOOSE
min_b = rr.MIN_B_VALUES[0]
max_b = rr.MAX_B_VALUES[-1]


combos = combinations(range(min_b, max_b + 1), b_count)
# print("generating combos: ", len(list(combos)))

In [17]:
lst = []
post_str_b1 = None
post_str_blast = None
for row in combos:
  cols = []
  str_row = " ".join([str(r).zfill(2) for r in row])
  id, bolillas_int, str_combo = formalize_data_with_str(bolillas_str=str_row)
  cols.append(id)
  cols.append(bolillas_int)
  cols.extend([b for b in bolillas_int])
  cols.extend(str_combo)
  cols.append(post_str_b1)
  cols.append(post_str_blast)

  is_rare, reason = rr.is_rare_combination(bolillas_int, id)
  cols.append(is_rare)
  cols.append(reason)

  cols.append(rr.jumps_map(bolillas_int))
  cols.append(rr.unijump(bolillas_int))
  
  lst.append(cols)

  post_str_b1 = str_combo[0]
  post_str_blast = str_combo[-1]

with open('./gn_cahe_v3.json', 'w') as file: 
  json.dump(lst, file)

df_Orows = pl.DataFrame(
  lst,
  schema=[
    # 'fecha', 'sorteo', 'bolillas',
    'id', 'combo', 'b1', 'b2', 'b3', 'b4', 'b5',
    'str_b1', 'str_b2', 'str_b3', 'str_b4', 'str_b5',
    'post_str_b1', 'post_str_blast',
    'is_rare_combination', 'reason',
    "jumps_map", "unijump"
  ],
  schema_overrides={
    'combo': pl.Array(pl.UInt8, 5),
    'b1': pl.UInt8,
    'b2': pl.UInt8,
    'b3': pl.UInt8,
    'b4': pl.UInt8,
    'b5': pl.UInt8,
    'b6': pl.UInt8,
    'reason': pl.UInt8
  },
  orient='row'
)

TOTAL_LINES = len(df_Orows)
print(f'{TOTAL_LINES=}')
display(df_Orows.head(10))

TOTAL_LINES=324632


id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""0102030405""","[1, 2, … 5]",1,2,3,4,5,"""01""","""02""","""03""","""04""","""05""",null,null,true,8,"""01_01_01_01""",0
"""0102030406""","[1, 2, … 6]",1,2,3,4,6,"""01""","""02""","""03""","""04""","""06""","""01""","""05""",true,8,"""01_01_01_02""",1
"""0102030407""","[1, 2, … 7]",1,2,3,4,7,"""01""","""02""","""03""","""04""","""07""","""01""","""06""",true,8,"""01_01_01_03""",2
"""0102030408""","[1, 2, … 8]",1,2,3,4,8,"""01""","""02""","""03""","""04""","""08""","""01""","""07""",true,8,"""01_01_01_04""",3
"""0102030409""","[1, 2, … 9]",1,2,3,4,9,"""01""","""02""","""03""","""04""","""09""","""01""","""08""",true,8,"""01_01_01_05""",4
"""0102030410""","[1, 2, … 10]",1,2,3,4,10,"""01""","""02""","""03""","""04""","""10""","""01""","""09""",true,8,"""01_01_01_06""",5
"""0102030411""","[1, 2, … 11]",1,2,3,4,11,"""01""","""02""","""03""","""04""","""11""","""01""","""10""",true,8,"""01_01_01_07""",6
"""0102030412""","[1, 2, … 12]",1,2,3,4,12,"""01""","""02""","""03""","""04""","""12""","""01""","""11""",true,8,"""01_01_01_08""",7
"""0102030413""","[1, 2, … 13]",1,2,3,4,13,"""01""","""02""","""03""","""04""","""13""","""01""","""12""",true,8,"""01_01_01_09""",8


In [18]:
df_Orows.filter(is_rare_combination=False)

id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""0102040513""","[1, 2, … 13]",1,2,4,5,13,"""01""","""02""","""04""","""05""","""13""","""01""","""12""",false,0,"""01_02_01_08""",10
"""0102040523""","[1, 2, … 23]",1,2,4,5,23,"""01""","""02""","""04""","""05""","""23""","""01""","""22""",false,0,"""01_02_01_18""",20
"""0102040530""","[1, 2, … 30]",1,2,4,5,30,"""01""","""02""","""04""","""05""","""30""","""01""","""29""",false,0,"""01_02_01_25""",27
"""0102040531""","[1, 2, … 31]",1,2,4,5,31,"""01""","""02""","""04""","""05""","""31""","""01""","""30""",false,0,"""01_02_01_26""",28
"""0102040532""","[1, 2, … 32]",1,2,4,5,32,"""01""","""02""","""04""","""05""","""32""","""01""","""31""",false,0,"""01_02_01_27""",29
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2829313435""","[28, 29, … 35]",28,29,31,34,35,"""28""","""29""","""31""","""34""","""35""","""28""","""35""",false,0,"""01_02_03_01""",-3
"""2830313334""","[28, 30, … 34]",28,30,31,33,34,"""28""","""30""","""31""","""33""","""34""","""28""","""35""",false,0,"""02_01_02_01""",-4
"""2830313435""","[28, 30, … 35]",28,30,31,34,35,"""28""","""30""","""31""","""34""","""35""","""28""","""35""",false,0,"""02_01_03_01""",-7


In [19]:
# possible_b1, exclude_jump_maps_b1

df_possible = df_Orows.filter(
  pl.col("str_b1").is_in(possible_b1),
  ~pl.col("jumps_map").is_in(exclude_jump_maps_b1 + exclude_jump_maps_blast),
  pl.col("str_b5").is_in(possible_blast),
  pl.col("is_rare_combination") == False
)

df_possible

id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""0102040530""","[1, 2, … 30]",1,2,4,5,30,"""01""","""02""","""04""","""05""","""30""","""01""","""29""",false,0,"""01_02_01_25""",27
"""0102040531""","[1, 2, … 31]",1,2,4,5,31,"""01""","""02""","""04""","""05""","""31""","""01""","""30""",false,0,"""01_02_01_26""",28
"""0102040532""","[1, 2, … 32]",1,2,4,5,32,"""01""","""02""","""04""","""05""","""32""","""01""","""31""",false,0,"""01_02_01_27""",29
"""0102040533""","[1, 2, … 33]",1,2,4,5,33,"""01""","""02""","""04""","""05""","""33""","""01""","""32""",false,0,"""01_02_01_28""",30
"""0102040534""","[1, 2, … 34]",1,2,4,5,34,"""01""","""02""","""04""","""05""","""34""","""01""","""33""",false,0,"""01_02_01_29""",31
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""0928313334""","[9, 28, … 34]",9,28,31,33,34,"""09""","""28""","""31""","""33""","""34""","""09""","""35""",false,0,"""19_03_02_01""",-15
"""0929313234""","[9, 29, … 34]",9,29,31,32,34,"""09""","""29""","""31""","""32""","""34""","""09""","""33""",false,0,"""20_02_01_02""",-15
"""0929313235""","[9, 29, … 35]",9,29,31,32,35,"""09""","""29""","""31""","""32""","""35""","""09""","""34""",false,0,"""20_02_01_03""",-14


In [20]:
df_possible.sample(n=1)

id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""0513192428""","[5, 13, … 28]",5,13,19,24,28,"""05""","""13""","""19""","""24""","""28""","""05""","""27""",false,0,"""08_06_05_04""",-1
